In [ ]:
; If using Jupyter Lab, run
;   jupyter labextension install @jupyterlab/javascript-extension
; before using this notebook.
(ns problemset
     (:refer-clojure :only [])
     (:require
        [clojure.repl :refer :all]
        [metaprob.trace :as trace]
        [metaprob.builtin-impl :as impl]
        [metaprob.syntax :refer :all]
        [metaprob.builtin :refer :all]
        [metaprob.prelude :refer :all]
        [metaprob.context :refer :all]
        [metaprob.distributions :refer :all]
        [metaprob.interpreters :refer :all]
        [metaprob.inference :refer :all]
        [metaprob.compositional :as comp]
        [metaprob.examples.gaussian :refer :all]
        [metaprob.tutorial.jupyter :refer :all]))

(enable-inline-viz)

# Welcome to the Metaprob Problem Set!

Metaprob is a new, experimental language for probabilistic programming, embedded in Clojure. In this problem set, we'll take a bottom-up approach to introducing the language. We'll begin by exploring Metaprob's core data types, including the flexible "trace" data structure at the heart of the language's design. From there, we'll look at the probabilistic semantics of Metaprob programs, and tackle some small statistical inference problems. By the end of this problem set, we'll be querying a more sophisticated model of real US census data!

For easy reference, the following table of contents contains links to all the problem set exercises, divided by section. Please type your solutions inline.

# Table of contents 
1. [Metaprob values and syntax](#values)
    
2. [Probabilistic modeling: a few simple models](#probmodeling)
    - [Exercise 1: Gambler's fallacy](#ex-gamblers)
    
3. [Monte Carlo estimation for expectations](#mc-estimation)
    - [Exercise 2: Expectations of indicator functions](#ex-expectations)
    - [Exercise 3: Pitfalls of MC estimation](#ex-pitfalls)
    
4. [Constructing and manipulating traces](#constructing)
    - [Exercise 4: Binary search tree](#ex-binary)
    - [Exercise 5: Automatic sizing for trace visualizations](#ex-automatic)
    - [Challenge Exercise A: Delete subtrace](#ex-delete)
    - [Challenge Exercise B: All addresses with values](#ex-alladdresses)  
    
5. [Expectations under interventions](#expectations-interventions)
    - [Exercise 6: What queries do interventions answer?](#ex-interventions)
6. [Conditioning with rejection sampling](#conditioning-rejection)
    - [Exercise 7: Estimating running time for rejection sampling](#ex-estimating)
    - [Exercise 8: Plotting the posterior](#ex-plotting)
7. [Conditioning with likelihood weighting](#conditioning-likelihood)
8. [Putting it all together](#putting)
    - [Challenge Exercise C: Custom inference for circus brothers](#ex-custombrothers)

# 1. Metaprob values and syntax <a name="values"></a>
The Metaprob language is embedded in Clojure, and inherits several of Clojure's atomic datatypes: Clojure's numbers, keywords, booleans, symbols, and strings are all Metaprob values, too. Many of Clojure's functions for manipulating these values have Metaprob versions.

In [ ]:
(+ 5 7)

If you want to use a Clojure function that is not available in Metaprob, you can fully qualify the procedure's name.

In [ ]:
(clojure.core/str "Hello" " " "world")

(Any Clojure function can be used without modification in Metaprob. For reasons we'll cover a bit later, however, calling impure Clojure functions that have nondeterministic behavior (e.g., because they generate random values during their execution) must be done with care, in order not to break Metaprob's probabilistic semantics. Note that this restriction applies also to higher-order Clojure functions supplied with stochastic procedures as arguments.)

Metaprob also supports Clojure's lists, maps, and vectors. They can all be manipulated using a unified set of Metaprob accessors:

In [ ]:
(get [5 2 5 1] 1) ; index 1

# 2. Probabilistic modeling: a few simple models <a name="probmodeling"></a>

There are many interesting questions that can be phrased using the language of probability:
- How much can I expect my lifetime earnings to improve if I get a college degree?
- What is the probability that this treatment will work for this patient?
- With what confidence can we say that these two documents were produced by the same author?

Before we can even begin to answer these questions, though, we need a _model_: a (sometimes simplified) mathematical description of how we imagine the real-world processes behind these quesitons (the job market, human biology, authorship) actually work. 

One of the key insights in probabilistic programming is that writing a program to simulate a process is a good way to specify our probabilistic models.

A few examples will make this clearer.


To define a function, use the form
```
(define func-name
  (gen [arg1 arg2 ...]
    body))
```

### Scenario 1: A tricky coin
Suppose you are interviewing for a position at a financial firm, and your interviewer produces a coin. He flips the coin 100 times, and you observe 100 heads. What is the probability, he asks you, that the next flip, too, will come up heads?

Your answer will depend on your _model_ of the random process you're observing.

Here is one possible model:

In [ ]:
(define coin-model-1
    (gen []
         (replicate 100 (gen [] (flip 0.5)))))

In [ ]:
(coin-model-1)

In [ ]:
(coin-model-1)

We can use the clojure function `filter` to extract the sublist of `true` values, which correspond to heads.

In [ ]:
(clojure.core/filter (gen [x] x) [false true false false true])

The function `count-heads` takes a sample from one of our coin flip models, and returns the number of heads observed.

In [ ]:
(define count-heads
    (gen [flips]
         (count (clojure.core/filter (gen [x] x) flips))))

In [ ]:
(count-heads [false true false false true])

In [ ]:
(count-heads (coin-model-1))

In [ ]:
(count-heads (coin-model-1))

In [ ]:
(replicate 100
           (gen []
                (count-heads (coin-model-1))))

Here, we have used `replicate`, which takes a number (100, in this case) and a 0-argument procedure, and runs the procedure that many times, returning a list of the results from each run. In this model, we hypothesize that the underlying process at work is as follows: a fair coin is flipped 100 times. We are 100% sure that the coin is fair: hence the constant `0.5`.

Another model might not be so trusting of the interviewer:

In [ ]:
(define coin-model-2
    (gen []
         (define p (uniform 0 1))
         (replicate 100 (gen [] (flip p)))))

In [ ]:
(coin-model-2)

In [ ]:
(replicate 100
           (gen []
                (count-heads (coin-model-2))))

Here, we imagine that the interviewer randomly decided on the coin's weight (a number between 0 and 1) before we entered the room, and procured a biased coin to flip for us.

Perhaps we don't really believe that's possible, though: how does one get a "weighted coin" anyway? Maybe we really believe only three possibilities going in: the coin is fair, a "heads" is painted on both sides, or a "tails" is painted on both sides. Let's try to model that scenario:

In [ ]:
(define coin-model-3
    (gen []
         (define p (uniform-sample [0 0.5 1]))
         (replicate 100 (gen [] (flip p)))))

In [ ]:
(coin-model-3)

In [ ]:
(replicate 100
           (gen []
                (count-heads (coin-model-3))))

In addition to the functions `define` and `filter`, you may find it helpful to use the functions `range`, `map`, or `reduce`, which we briefly illustrate:

In [ ]:
(range 50)

In [ ]:
(map (gen [x] (+ x 2)) (range 20))

In [ ]:
(map (gen [x] true) (range 20))

In [ ]:
(clojure.core/reduce + (range 10))

### Exercise 1: Gambler's fallacy <a name="ex-gamblers"></a>

The "gambler's fallacy" is (according to Wikipedia) "the mistaken belief that, if something happens more frequently than normal during a given period, it will happen less frequently in the future." Create a model `coin-model-4` that implements some cognitively plausible mechanism by which the weight of the coin is adjusted after each flip to help ensure, at every step, that the whole _sequence_ of flips will be more evenly split between Heads and Tails than actually happens by chance with a fair coin.

In [ ]:
; Your solution here

(define coin-model-4
    (gen []
         '...))

In [ ]:
; Solution

(define coin-model-4
    (gen []
         (reduce 
             (gen [l n]
                  (define current-ratio (/ (count (clojure.core/filter (gen [x] x) l)) n))
                  (cons (flip (- 1 current-ratio)) l))
             (list (flip 0.5))
             (map (gen [x] (+ x 1)) (range 99)))))

In [ ]:
(coin-model-4)

In [ ]:
(replicate 100
           (gen []
                (count-heads (coin-model-4))))

### Scenario 2: Sea animals

The following is an inaccurate and highly simplified model of sea animals, in which every sea animal is either a starfish or octopus, there are roughly equal numbers of each, and the number of limbs is a continuous quantity (e.g., because there can be a fraction of a tentacle or leg remaining).

Octopuses generally are heavier than starfish, and for either species, animals with more limbs are heavier. Of course, species is correlated with the number of limbs: octopuses tend to have more limbs than starfish.

We may want to answer questions like:
- If all we know about about a sea animal is that it has 7 limbs, what might we expect its weight to be? Its species?
- What is the probability that a starfish's weight will be within some range?
- and many more...

We can encode the data on the subject into a model, where `species` is either octopus or starfish, `limbs` represents the number of limbs, and `weight` represents its weight in grams.

In [ ]:
(define sea-animals-model
  (gen []
    (define species (uniform-sample [:octopus :starfish]))
    (define limbs-mean (if (= species :octopus) 8 6))
    (define limbs (gaussian limbs-mean 1))
    (define base-weight-mean (if (= species :octopus) 2400 1700))
    (define limbs-adjusted-weight-mean (* base-weight-mean (exp (* (- limbs limbs-mean) (if (= species :octopus) 0.024 0.019)))))
    (define weight (gaussian limbs-adjusted-weight-mean 100))
    weight))

In [ ]:
(sea-animals-model)

In [ ]:
(replicate 10 (gen [] (sea-animals-model)))

# 3. Monte Carlo estimation for expectations <a name="mc-estimation"></a>

Many queries can be formed as questions about averages. For example, we might wonder, under our sea animals model, what the average weight is (regardless of species or number of limbs). Or, under our various coin flip models, we might ask what the average number of heads is. More interestingly, we can also ask about averages under some condition: given that the first ten flips were heads, what is the average total number of heads?

In these contexts, the word "average" is not quite accurate. We can take averages of actual lists of numbers, but our models are not lists of numbers: they are stochastic processes that _produce_ numbers (and other data).

What we are really asking about is something called an "expectation." If we have a generative probabilistic model `p`, and a function `f` that takes in samples from `p` and outputs numbers, the _expectation of `f` with respect to the distribution `p`_ is the weighted average of the values taken by `(f x)` as `x` is drawn from the distribution `p`—we weight more probable values of `x` more heavily, and less probable values more lightly. The expectation can be obtained (with probability 1) by taking the limit of the average, as our number of samples goes to infinity, of samples of `(f (p))`.

Our earlier function `count-heads` can play the role of `f`.

To view the source code of this and other functions we have seen, we can use `:generative-source` as follows:

In [ ]:
(get replicate :generative-source)

In [ ]:
(get coin-model-1 :generative-source)

In [ ]:
(get count-heads :generative-source)

In [ ]:
(count-heads (coin-model-1))

In [ ]:
(count-heads (coin-model-2))

In [ ]:
(count-heads (coin-model-3))

Above, we've taken one sample of `(f (p))` for each coin flip model `p`. What we'd like to do is take the expectation -- the expected average over infinitely many samples. There are ways to calculate this number exactly; indeed, you probably have a good intuition for what it should be in the three cases above. But in more complex models, that can be harder, so we will focus here on estimation.

The simplest technique for estimating an expectation is taking a lot of samples and averaging them. Let's write that up:

In [ ]:
(define avg (gen [l] (clojure.core/float (/ (apply + l) (count l)))))

(define mc-expectation
    (gen [p f n]
         (avg (map f (replicate n p)))))

In [ ]:
(clojure.core/meta coin-model-3)

In [ ]:
(get coin-model-2 :generative-source)

In [ ]:
(get coin-model-1 :generative-source)

In [ ]:
(get mc-expectation :generative-source)

In [ ]:
(mc-expectation coin-model-1 count-heads 1000)

In [ ]:
(mc-expectation coin-model-2 count-heads 1000)

In [ ]:
(mc-expectation coin-model-3 count-heads 1000)

### Exercise 2: Expectations of indicator functions <a name="ex-expectations"></a>

We can also use this simple technique to ask about _probabilities_ of events. For example, to estimate the probability under each model of attaining at least 75 heads, we can estimate the expectation of a function that returns one when the condition holds, and zero otherwise. We call this an _indicator_ function.

Write a function `indicator` which takes in a `condition` (a predicate that accepts or rejects a given value) and returns an indicator function for that condition (a function that returns 0 or 1 based on whether its input satisfies the condition). Use this to create an indicator for the condition in which there are more than 75 heads in an experiment, and estimate the probability of this event under coin models 1, 2, and 3.

In [ ]:
; Your solution here

(define indicator
    (gen [condition] '...))

In [ ]:
; Solution:

(define indicator
    (gen [condition]
         (gen [x] (if (condition x) 1 0))))

In [ ]:
(mc-expectation coin-model-1 (indicator (gen [l] (>= (count-heads l) 75))) 2000)

In [ ]:
(mc-expectation coin-model-2 (indicator (gen [l] (>= (count-heads l) 75))) 2000)

In [ ]:
(mc-expectation coin-model-3 (indicator (gen [l] (>= (count-heads l) 75))) 2000)

### Exercise 3: Pitfalls of MC estimation <a name="ex-pitfalls"></a>

Under what conditions is `mc-expectation` likely to give a misleading answer? (Hint: under `coin-model-1`, is the probability of 75 heads really 0? What is the expected amount won in a lottery?)

In [ ]:
; Your solution here

In [ ]:
; Solution

; [To be added]

Let's try to apply our `mc-expectation` procedure to our sea animals model, first to answer the question, "Under our model, what is the average person's height?"

We immediately hit a snag: last time, the number of heads was a straightforward transformation of `coin-model-i`'s return value. Here, `sea-animals-model` returns only a weight, from which we cannot recover the number of limbs of the sea animal whose weight it is.

We could alter the model code to return more values—say, `[species limbs weight]` as a vector—but there's another way.

So far, we've seen only one way of interacting with a Metaprob procedure: running it. But Metaprob provides a lot more flexibility than that, via its powerful `infer` operator.

One of the most basic things we can ask `infer` to do for us is _trace the random choices_ made by a Metaprob procedure:

In [ ]:
(infer
    :procedure sea-animals-model)

`infer` runs the given procedure and returns three values: the procedure's return value (in this case, a weight), an _execution trace_ that records the random choices made during the procedure's execution, and a "score" (which is zero in this case — we'll ignore it for now).

Let's look closer at the second return value—the execution trace.

In [ ]:
(define [weight tr _]
    (infer
        :procedure sea-animals-model))
(plot-trace tr)

The function `sea-animals-model` makes three random choices during its execution, and as such, its execution trace records three values. The addresses at which they are stored reflect where in `sea-animals-model`'s source code they were made. For example, the first recorded value is the choice `:f`, at address `'(0 "species" "uniform-sample")`. This indicates that the choice in question was made on line 0, while defining the variable `species`, using the procedure `uniform-sample`.

Let's rewrite our `mc-expectation` procedure to run `f` on the _execution trace_ of `p`, rather than its return value:

In [ ]:
(define mc-expectation-v2
    (gen [p f n]
         (avg
             (map f (replicate n
                               (gen []
                                    (define [_ tr _] (infer :procedure p)) tr))))))

We can now answer our question about average number of limbs:

In [ ]:
(define sea-animal-limbs
  (gen [t] (trace-value t '(2 "limbs" "gaussian"))))

In [ ]:
(mc-expectation-v2 sea-animals-model sea-animal-limbs 3)

This is quite close to the true answer.

It can be cumbersome to type the addresses manually. Let's give them short names:

In [ ]:
(addresses-of tr)

In [ ]:
(define [species-adr limbs-adr weight-adr] (addresses-of tr))

Estimating an expectation can be useful, but one of the advantages of Bayesian inference is that we can also quantify our uncertainty. By taking a lot of samples and plotting them in a histogram, we can get a sense of what an entire distribution looks like.

In [ ]:
(histogram "Model 1" 
           (map count-heads (replicate 1000 coin-model-1)) [0 100] 1)

In [ ]:
(histogram "Model 2" (map count-heads (replicate 100 coin-model-2)) [0 100] 1)

In [ ]:
(histogram "Model 3" (map count-heads (replicate 400 coin-model-3)) [0 100] 1)

# 4. Constructing and manipulating traces <a name="constructing"></a>

We have already seen one execution trace. Let's look at some of the tools Metaprob provides for manipulating and creating traces.

We'll start with trace _manipulation_. We'll use the trace depicted above as a running example. Let's give it a name using the Metaprob keyword `define`. Below, we also demonstrate the `pprint` function, which is another way (apart from `plot-trace`) to get a human-readable representation of a trace. You can ignore the word `COMPILED` for now: it just indicates that the Metaprob has produced an efficient Clojure-compiled version of the procedure that this trace represents (something the `gen` macro is also responsible for).

A trace is a tree-like data structure. Every trace
* has a root node (the leftmost gray box in the diagrams above);
* optionally stores a value at the root node (displayed inside that gray box); and
* has zero or more named subtraces (connected to the root by thin edges in the diagrams above). Each subtrace has its own root node and, potentially, its own named subtraces.

Subtraces may be named with strings, numbers, or booleans. The name of each subtrace appears above its root node in the diagrams above.

## 4.1 Reading values in traces

One of the most basic things we can do with a trace is get its root value, using `trace-value`:

In [ ]:
(trace-value {:value 10})

Not all traces have values at their roots; it is sometimes useful to call `trace-has-value?`, which checks if a trace has a root value.

In [ ]:
; A vector is represented as a trace with no value at its root,
; and with subtraces for each of its elements.
(trace-has-value? [1 2 3])

There is an optional second argument to `trace-has-value?`: an _address_. An address is either:
- the name of a subtrace, or
- a list of names, representing a "path" to the desired subtrace.

This version of `trace-has-value?` returns false if the given address is invalid for the trace, or if it is valid but the specified subtrace has no value at its root.

## 4.2 Setting (and clearing) values and subtraces

Metaprob provides two functions, `(trace-set tr [adr] val)` and `(trace-set-subtrace tr adr sub)`, which return modified versions of traces:
- `(trace-set tr val)` changes the value of the trace `tr`'s root node to `val`.
- `(trace-set tr adr val)` changes the _root value_ of the subtrace at the address `adr` to `val`.
- `(trace-set-subtrace tr adr sub)` changes the entire subtrace of `tr` at `adr` to `sub`.

These functions do not change the original trace, but rather create a _copy_ of the trace with some value or subtrace changed. Metaprob does have mutable traces, and operators like `trace-set!` and `trace-set-subtrace!` that operate on them, but we will not have a need for them in this tutorial.

## 4.3 Constructing traces

In [ ]:
(plot-trace {} 100)

One way to construct a bigger trace is to start out with an empty one and use `trace-set` and `trace-set-subtrace` to add values and sub-traces. This can be quite tedious, so `(trace ...)` also supports an argument syntax that is demonstrated in the examples below:

In [ ]:
; Trace with a root value
(plot-trace { :value 10} 75)

In [ ]:
; Trace with subtraces with root values
(plot-trace {"Massachusetts" {:value "Boston"}, 
             "California" {:value "Sacramento"}, 
             "New York" {:value "New York"}} 300)

In [ ]:
; Trace with root value and subtraces with root values
(plot-trace
    {:value "John Doe",
     "first name" {:value "John"}, 
     "last name" {:value "Doe"}, 
     "address" {:value "500 Penn Ave."}, 
     "age" {:value 94}}
    400)

In [ ]:
; Trace with arbitrary named subtrace, using (** ...)
(plot-trace
    {"a" {"b" {:value "c"}},
     "b" {:value 5}} 
  300 100)

In [ ]:
; Trace with multiple named subtraces and a value
(plot-trace
    {:value "phrases", 
     "green" {:value "first word",
              "eggs" {"and" {"ham" {:value "!"}}},
              "peace" {:value "."},
              "ideas" {"sleep" {"furiously" {:value "?"}}}}})

### Exercise 4: Binary search tree <a name="ex-binary"></a>

As a way to get used to these trace operations, in this exercise, we'll implement binary search trees in Metaprob using traces. For our purposes, a binary search tree is either:
- The empty trace (representing an empty tree)
- A trace with a number at its root, and (optionally) "left" and "right" subtraces which are themselves binary trees. All values in the left subtrace should be less than or equal to the root's value, and all values in the right subtrace should be greater than or equal to the root's value.

For this exercise, write:
1. A manually constructed binary search tree, using the `(trace ...)` function, with the numbers 1 through 10. (There are multiple valid binary search trees containing these numbers; any organization that is consistent with the above rules is fine.)
2. A function `(insert tree i)`, which inserts a number `i` into `tree`.
3. A function `(contains? tree i)` which checks whether a tree contains a given node.

In [ ]:
; Your solution here

(define example-tree
    {:value '...})

(define insert 
    (gen [tree i]
         '...))

(define tree-contains?
    (gen [tree i]
         '...))

In [ ]:
; Solution

(define example-tree
    {:value 5
    "left" {:value  3,
            "left" {:value 1 
                    "right" {:value 2}},
            "right" {:value 4}},
    "right" {:value  7,
             "left" {:value 6},
             "right" {:value 9,
                      "left" {:value 8},
                      "right" {:value 10}}}})
(define insert 
    (gen [tree i]
         (if (empty? tree)
             (trace-set-value tree i)
             (block
                 (define branch (if (<= i (trace-value tree)) "left" "right"))
                 (trace-set-subtrace
                     tree
                     branch
                     (insert (if (trace-has-subtrace? tree branch)
                                 (trace-subtrace tree branch)
                                 {}) i))))))

(define tree-contains?
    (gen [tree i]
         (and (not (empty? tree))
              (or
                (= (trace-value tree) i)
                (block
                  (define branch (if (<= i (trace-value tree)) "left" "right"))
                  (and (trace-has-subtrace? tree branch)
                       (tree-contains? (trace-subtrace tree branch) i)))))))

In [ ]:
(clojure.core/every? (gen [n] (tree-contains? example-tree (+ n 1))) (range 10))

## 4.5 Traversing traces

You've now written several functions that traverse traces recursively, but you always knew the general structure of the trace you were traversing. For cases when you don't know ahead-of-time how many children a trace will have -- or what its subtraces will be called -- the `(trace-keys tr)` function comes in handy. It lists the names of all subtraces of a given trace.

In [ ]:
(trace-keys example-tree)

Note that Metaprob has a version of Clojure's `map` function, which often comes in handy when dealing with lists of subtrace names:

Metaprob also has `apply`, which applies a procedure to a list of arguments:

In [ ]:
; Concatenate the names of a trace's children
(apply str (trace-keys example-tree))

### Exercise 5: Automatic sizing for trace visualizations <a name="ex-automatic"></a>


Currently, the `plot-trace` function draws trace diagrams of whatever size the user specifies. In this exercise, you'll write a function `smart-plot-trace` that automatically decides on a size for the trace, based on its breadth and depth:

1. Begin by defining `(trace-depth tr)`, which recursively computes the maximum depth of a trace.
2. Next, define `(trace-breadth tr i)`, which gives the trace's breadth at level `i`. (Hint: sum the widths of the sub-traces at level `i-1`.)
3. Define `(max-trace-breadth tr)` which gives the maximum breadth of a trace at any of its levels. (Note: the implementation we are suggesting in this exercise, which simply calls `trace-breadth` once for each level of the trace, is $O(n^2)$; there are certainly more efficient algorithms!)
4. Write a function `smart-plot-trace` that calls `plot-trace`, passing in reasonable values for diagram width and height (based on the depth and breadth of the trace).

In [ ]:
; Bring in some useful Clojure functions/macros
(clojure.core/refer-clojure :only '[max])

In [ ]:
; Your solution here
(define trace-depth
    (gen [tr]
         '...))

(define trace-breadth
   (gen [tr i]
         '...))

(define max-trace-breadth
   (gen [tr]
         '...))

(define smart-plot-trace
   (gen [tr]
         '...))

In [ ]:
; Solution
(define trace-depth
  (gen [tr]
    (if (empty? (trace-keys tr))
        1
        (apply max 
            (map (gen [k]
                     (+ 1 (trace-depth (trace-subtrace tr k)))) (trace-keys tr))))))

(define trace-breadth
   (gen [tr i]
     (if (= i 0)
         1
         (apply 
             + 
             (map (gen [k]
                     (trace-breadth (trace-subtrace tr k) (- i 1))) (trace-keys tr))))))

(define max-trace-breadth
   (gen [tr]
     (apply max (map (gen [i] (trace-breadth tr i)) (range (trace-depth tr))))))

(define smart-plot-trace
   (gen [tr]
     (plot-trace tr (* (trace-depth tr) 100)
                    (* (max-trace-breadth tr) 80))))

In [ ]:
(smart-plot-trace example-tree)

In [ ]:
(infer :procedure (gen [x] (flip x)) :inputs [0.5])

In [ ]:
(infer :procedure (gen [x] (and (flip x) (flip x))) :inputs [0.5])

In [ ]:
(infer :procedure comp/infer-apply :inputs [(gen [] (flip 0.5)) [] (make-top-level-tracing-context {} {})])

In [ ]:
(smart-plot-trace
  (nth (infer :procedure comp/infer-apply :inputs [(gen [] (flip 0.5)) [] (make-top-level-tracing-context {} {})]) 1))

### Challenge Exercise A: Delete subtrace <a name="ex-delete"></a>

Write a function `(delete-subtrace tr adr)` that completely deletes the addressed subtrace from a trace.

In [ ]:
; Your solution here

In [ ]:
; Solution

; [To be added]

### Challenge Exercise B: All addresses with values <a name="ex-alladdresses"></a>


Write a function `(all-addresses-with-values tr)` which returns a list of addresses at which `tr` has a value. (The built-in function `addresses-of` can already do this.)

In [ ]:
(clojure.core/refer-clojure :only '[filter])

In [ ]:
(define trace-set-values-at
  (gen [t & pairs]
    (if (empty? pairs)
        t
        (block
          (define addr (first pairs))
          (define value (first (rest pairs)))
          (define others (rest (rest pairs)))
          (apply trace-set-values-at (cons (trace-set-value t addr value) others))))))

In [ ]:
; Your solution here

In [ ]:
; Solution

; [To be added]

# 5. Expectations under interventions <a name="expectations-interventions"></a>

Although we can sometimes learn interesting things by taking expectations with respect to our models, the most exciting questions usually require _conditioning_ on some piece of data. We might wonder what a sea animals's expected weight is _given that_ it is a starfish, or with what probability a sea animal is a starfish _given that_ it weighs 1800 grams.

Let's look at the simpler of those two questions first. One way to estimate an answer would be to run many simulations in which we force the model to assign to `species` the value `:starfish`, then average the resulting salaries. 

We can _intervene on_ our model and force it to make a certain choice by passing an _intervention trace_ to `infer`. `infer` runs the model as usual, but when it encounters a random choice with an address that appears in the intervention trace, instead of generating a new choice, it simply reuses the one in the intervention trace. 

In the code below, notice the modified call to `infer`, and also the manner in which we construct an intervention trace using the address of the choice we wish to control:

In [ ]:
; First, create a version of mc-expectation that accepts an intervention trace
(define mc-expectation-v3
    (gen [p intervene f n]
         (avg
             (map f (replicate n
                               (gen []
                                    (define [_ tr _] (infer
                                                         :procedure p
                                                         :intervention-trace intervene))   ; NEW
                                    tr))))))

; Create our intervention trace
(define ensure-starfish
    (trace-set-value {} species-adr :starfish))

; Create our accessor (the f to take the expectation of)
(define get-weight
    (gen [t] (trace-value t weight-adr)))

; Run the query
(mc-expectation-v3 sea-animals-model ensure-starfish get-weight 4)

This is quite a limited technique, however. If we wanted to ask, for instance, about a sea animal's expected number of limbs, given that their weight is 1200 grams, an intervention would not do the trick. Do you see why? Here's what that (wrong) code would look like:

In [ ]:
; Create our intervention trace
(define ensure-1200
  (trace-set-value {} weight-adr 1200))

; Create our accessor (the f to take the expectation of)
(define get-limbs
   (gen [t] (trace-value t limbs-adr)))

; Run the query
(mc-expectation-v3 sea-animals-model ensure-1200 get-limbs 1000)

In [ ]:
; Run the query with NO intervention — just calculates average number of limbs, regardless of weight
(mc-expectation-v3 sea-animals-model {} get-limbs 1000)

### Exercise 6: What queries do interventions answer? <a name="ex-interventions"></a>

Why are these two numbers (almost) the same? Can you characterize the situations when interventions _do_ work as a method of conditioning? Write code for one conditional query that can be answered with interventions, and one that can't.

In [ ]:
; Your solution here

In [ ]:
; Solution

; [To be added]

# 6. Conditioning with rejection sampling <a name="conditioning-rejection"></a>

In order to answer more sophisticated conditional queries, we need to turn to another technique: rejection sampling. In rejection sampling, we run our model over and over until we get a sample that satisfies our condition. To estimate a _conditional expectation_ using rejection sampling, we can do this $n$ times, and average the values `f` takes on the $n$ samples that satisfy our condition.

In [ ]:
(define rejection-sample
    (gen [p condition f]
         (define [_ t _] (infer :procedure p))
         (if (condition t) (f t) (rejection-sample p condition f))))

(define rejection-expectation
    (gen [p condition f n]
         ; Define a modified version of p which tries again
         ; if its first execution doesn't satisfy the condition.
         ; Call it n times
         (avg (replicate n (gen [] (rejection-sample p condition f))))))

Now, suppose we want to know the expected number of limbs of a sea animal weighing 2800 grams — a very high weight under our model. If we expressed our condition as "a weight of exactly 2800 grams", it would take a very long time to get even one sample. So instead, we ask about a small interval around 2800 grams. It is still quite slow:

In [ ]:
(rejection-expectation
    sea-animals-model
    (gen [t] (< 2700 (trace-value t weight-adr) 2900))
    (gen [t] (trace-value t limbs-adr))
    100)

If we make the same query, but for an interval around 2000 grams, it runs much faster:

In [ ]:
(rejection-expectation
    sea-animals-model
    (gen [t] (< 1900 (trace-value t weight-adr) 2100))
    (gen [t] (trace-value t limbs-adr))
    100)

This is because our new condition is much more probable. As such, it takes fewer "tries" to produce a sample that satisfies it.

### Exercise 7: Estimating running time for rejection sampling <a name="ex-estimating"></a>

Write two rejection sampling queries to estimate (a) the probability that a sea animal weighing between 2000 and 2100 grams is a starfish, and (b) the average number of limbs of a sea animal whose weight is within that range. Before you run each one, try to estimate (roughly, qualitatively) how fast or slow it will be.

In [ ]:
; Your solution (a) here
(rejection-expectation
    sea-animals-model
    '...
    '...
    100)

In [ ]:
; Your solution (b) here
(rejection-expectation
    sea-animals-model
    '...
    '...
    100)

In [ ]:
; Solution (a).
(rejection-expectation
    sea-animals-model
    (gen [t] (< 2000 (trace-value t weight-adr) 2100))
    (indicator (gen [t] (= (trace-value t species-adr) :starfish)))
    100)

In [ ]:
; Solution (b). (slower)
(rejection-expectation
    sea-animals-model
    (gen [t] (and (= (trace-value t species-adr) :starfish) (< 2000 (trace-value t weight-adr) 2100)))
    (gen [t] (trace-value t limbs-adr))
    100)

With rejection sampling, we can begin to see the first applications of probabilistic modeling to _data analysis_. The idea is that we _condition_ our model's execution on the actual data we've observed, then ask questions about the _latent variables_ — the pieces we may not have observed directly.

As one example, consider the following model:

In [ ]:
(define hybrid-coin-model
    (gen []
         (define which-model (uniform-sample [coin-model-1 coin-model-2 coin-model-3]))
         (which-model)))

Given that we saw 100 heads, we can now ask about likely values of `which-model`. This allows us to pose questions like: if I saw 100 heads, which of my three possible explanations was most likely? And what are the chances the coin comes up heads next time?

In [ ]:
(define count-heads-in-trace
    (gen [t]
         (apply + (map (gen [adr] (if (and (boolean? (trace-value t adr)) (trace-value t adr)) 1 0))
                       (addresses-of t)))))

In [ ]:
(hybrid-coin-model)

In [ ]:
; Probability of Coin Model 3 if I saw 100 heads:
(rejection-expectation
    hybrid-coin-model
    (gen [t] (= (count-heads-in-trace t) 100))
    (indicator (gen [t] (= coin-model-3 (trace-value t '(0 "which-model" "uniform-sample")))))
    100)

In [ ]:
; Estimated probability of next coin flip being heads if I saw 100 heads
(rejection-expectation
    hybrid-coin-model
    (gen [t] (= (count-heads-in-trace t) 100))
    (gen [t]
      (define actual-model (trace-value t '(0 "which-model" "uniform-sample")))
      (cond
        (= actual-model coin-model-1) 0.5
        (= actual-model coin-model-2) (trace-value t '(1 "which-model" 0 "p" "uniform"))
        (= actual-model coin-model-3) (trace-value t '(1 "which-model" 0 "p" "uniform-sample"))))
    100)

This is (in most runs) _almost_ 1, but indicates that there is still some uncertainty.

## 6.1 Aside: The posterior distribution

So far, we have been making "point estimates" to answer our queries, summarizing our answers as a single number. For example, suppose suppose I tell you I have flipped a coin 100 times and it came up heads more than sixty times. I ask you to guess how many times exactly it came up heads. Using the `hybrid-coin-model`, you could compute an expectation, which is one way to generate a single-number answer:

In [ ]:
(rejection-expectation
    hybrid-coin-model
    (gen [t] (> (count-heads-in-trace t) 60))
    count-heads-in-trace
    100)

But one of the main advantages of the sort of probabilistic modeling we're doing is that we have access to much richer information. 

To introduce some terminology, the process of "Bayesian data analysis" consists of the following steps:

*Step 1*. First, express our prior beliefs about a generative process by encoding them into a model. The distribution over traces induced by that model is called the _prior_: it encodes our beliefs _prior_ to seeing any data. In our case, the prior distribution over "number of heads" looks like this:


In [ ]:
(histogram "Prior" (replicate 500 (gen [] (count-heads (hybrid-coin-model)))) [0 100] 1)

As we can see by examining this plot, although we believe that any number of heads is theoretically possible, we would be much more surprised to see, say, 75 heads than 0, 50, or 100. We can also plot our prior beliefs about which of the three models is being used, and check that all three are basically equally likely:

In [ ]:
(define model-number
 (gen [t] 
      (define which (trace-value t '(0 "which-model" "uniform-sample")))
      (cond 
        (= which coin-model-1) 1
        (= which coin-model-2) 2
        (= which coin-model-3) 3)))

(histogram "Prior on model choice" (replicate 500 (gen [] (model-number (nth (infer :procedure hybrid-coin-model) 1)))) [1 3] 1)

*Step 2.* Condition on observed data, retrieving the _posterior distribution_ -- like the prior distribution, but updated to reflect our new beliefs after seeing the data. In this case, we can observe that over 60 coins came up heads:

In [ ]:
(define plot-posterior-rejection
  (gen [p condition f n [min-val max-val] step]
    (histogram "Posterior" (replicate n (gen [] (rejection-sample p condition f))) [min-val max-val] step)))

In [ ]:
(plot-posterior-rejection hybrid-coin-model (gen [t] (> (count-heads-in-trace t) 60)) count-heads-in-trace 100 [0 100] 1)

As we can see, our point estimate above will in most cases be quite misleading!

We can also plot the posterior on `which-model`:

In [ ]:
(plot-posterior-rejection hybrid-coin-model (gen [t] (> (count-heads-in-trace t) 60)) model-number 500 [1 3] 1)

### Exercise 8: Plotting the posterior <a name="ex-plotting"></a>

Write a query that plots the weight distribution for sea animals with more than 8 limbs:

In [ ]:
; Your solution here

In [ ]:
; Solution
(plot-posterior-rejection
  sea-animals-model
  (gen [t] (> (trace-value t limbs-adr) 8))
  (gen [t] (trace-value t weight-adr))
  300
  [1000 2000] 30)

# 7. Conditioning with likelihood weighting <a name="conditioning-likelihood"></a>

In the last two sections, we saw two ways of generating _conditional_ samples:

1. Intervening on our model to force the "random choices" of interest to take on the values we want them to. This is fast, but is only correct in select cases, where the random choices we're conditioning on don't depend on any prior random choices.

2. Rejection sampling. This gives exact samples from the posterior; the only problem is how slow it is.

We'd like the speed of (1) with the correctness of (2). As a step toward getting there, let's think a bit more about why exactly (1) is usually incorrect.

Suppose we are trying to estimate the expected weight for a sea animal that has 8 limbs. We can get a (slow) estimate via rejection sampling:

In [ ]:
(rejection-expectation
    sea-animals-model
    (gen [t] (< 7.9 (trace-value t limbs-adr) 8.1))
    (gen [t] (trace-value t weight-adr))
    100)

If we do the same using our intervention trace method, we get a fast but inaccurate answer:

In [ ]:
(mc-expectation-v3
    sea-animals-model
    (trace-set-value {} limbs-adr 8)
    (gen [t] (trace-value t weight-adr))
    100)

Why is the answer lower in the intervention trace method? Under our model, octopuses are more likely to have 8 limbs than are starfish, so if we know that a sea animal has 8 limbs, we might expect it's more likely for it to be an octopus, and—incorporating the fact that octopuses weigh more even with the same number of limbs—we might expect its weight to be higher. The rejection sampling algorithm takes this into account. It simply waits for 100 samples where the number of limbs was 8, the majority of which will be octopuses; the weight will have been higher for those samples, so the expected weight will be higher.

The intervention method, on the other hand, is impatient, and immediately sets the number of limbs to 8 in every run, severing the tie between species and number of limbs. That means that our collection of 100 samples will contain just as many 8-limbed starfish as 8-limbed octopuses, which is improbable under our model.

What if we made a compromise? We still artificially set the number of limbs of each sample to 8, but also measure how likely the model _would have been_ to choose 8 anyway—a number that will be higher for `species=:octopus` samples than for `species=:starfish` samples. (For those familiar with probability theory, we're talking about probability densities: `p(limbs=8|species)`.) At the end, we take a _weighted average_ of the weights, where each weight is weighted by the likelihood that the sea animal really would have had 8 limbs (given the random choices we'd already made about them: their species).

To implement this, `infer` has one final trick up its sleeve: we can provide a _target trace_, which works just like an intervention trace, but also causes the constrained choices to be _scored_. For each targeted choice, Metaprob computes the probability that it _would have_ made the same choice under the prior, and accumulates these numbers (as a sum of _log_ probabilities) into a _score_ that is returned, along with the model's output and execution trace, from `infer`.

We can use this feature to implement the likelihood weighting algorithm:

In [ ]:
; Note -- we exponentiate the scores, because they are returned as log probabilities
(define weighted-samples
    (gen [proc target f n]
         (define samples (replicate n (gen [] (infer :procedure proc :target-trace target))))
         (map (gen [[o t s]] 
                   [(f t) (exp s)]) samples)))

(define lh-weighting
    (gen [proc target f n]
         (define samples (weighted-samples proc target f n))
         (/ (apply + (map (gen [[v s]] (* v s)) samples))
            (apply + (map (gen [[_ s]] s) samples)))))

Let's apply the technique to the question we explored above, to estimate the expected weight for a sea animal with 8 limbs:

In [ ]:
(lh-weighting sea-animals-model
              (trace-set-value {} limbs-adr 8)
              (gen [t] (trace-value t weight-adr))
              100)

This brings us quite close to the rejection-sampled estimate, and much faster!

## 7.1 Sampling/importance resampling
This same idea can be used to produce (and plot) samples from the posterior. The trick is this: to generate a single sample from the (approximate) posterior, we sample some number $n$ of "particles" with likelihood weights. We then choose one of them at random, giving each one probability proportional to its score. This variant of the technique is called "sampling/importancee resampling." Let's implement it:

In [ ]:
(define sample-importance-resample
    (gen
        [proc target f n-particles]
        ; Generate particles
        (define particles (weighted-samples proc target f n-particles))
        ; Choose one at random, with prob. proportional to importance weight
        (define sum-of-weights (apply + (map (gen [[_ s]] s) particles)))
        (define which-particle (categorical (map (gen [[o s]] (/ s (+ 1e-10 sum-of-weights))) particles)))
        ; Return only the sampled value (not the score)
        (define [sampled-value _] (nth particles which-particle))
        sampled-value))

In [ ]:
(define plot-posterior-importance
    (gen [p target f particles-per-sample n [min-val max-val] step]
         (histogram "Posterior (Importance Resampling)" (replicate n (gen [] (sample-importance-resample p target f particles-per-sample))) [min-val max-val] step)))

One last time, we consider the problem of estimating a sea animal's weight given that they have 9 limbs. This time, instead of using a point estimate, we'll plot the entire posterior, which will give us a better sense of how weights might vary within the population of 9-limbed sea animals.

Let's first plot using only one particle. In this case, we are not getting the posterior at all: we are just sampling from the prior with an intervention (setting limbs-9).

In [ ]:
; 1 particle -- sampling from the (intervened) prior
(plot-posterior-importance 
    sea-animals-model
    (trace-set-value {} limbs-adr 9)
    (gen [t] (trace-value t weight-adr))
    1 1000
    [1000 3000] 50)

Using 5 particles per sample, we get a better approximation to the posterior:

In [ ]:
; 5 particles -- better approx to posterior
(plot-posterior-importance 
    sea-animals-model
    (trace-set-value {} limbs-adr 9)
    (gen [t] (trace-value t weight-adr))
    5 500
    [1000 3000] 50)

To check our work, we can revert to the rejection sampling technique to get exact samples from the posterior:

In [ ]:
(plot-posterior-rejection
    sea-animals-model
    (gen [t] (< 8.9 (trace-value t limbs-adr) 9.1))
    (gen [t] (trace-value t weight-adr))
    500
    [1000 3000] 50)

This looks pretty similar. The upshot is that the sampling/importance resampling method gives us an accurate picture of the posterior in a much more efficient manner.

# 8. Putting it all together <a name="putting"></a>

In this section, we'll review all the techniques we've learned on a new example model, and look at ways of extending our models with model-specific "custom inference procedures."

The model we're using in this section is a _bivariate Gaussian_. A bivariate Gaussian models pairs of variables that each follow a "bell curve," but are correllated with one another — for example, a person's height and weight. 

Ignoring weight, people's heights are distributed normally, with a high probability at the mean and lower probability the further you get away from the mean. Here we plot samples from a _univariate_ Gaussian with mean 70 and _standard deviation_ 3; standard deviation is a measure of the "spread" of a distribution.

In [ ]:
(histogram-with-curve
    "Gaussian (height)"
    (replicate 5000 (gen [] (gaussian 70 3)))
    (gen [x] (exp (score-gaussian x [70 3])))
    [50 90])

And weight may also be distributed normally, with different mean and standard deviation:

In [ ]:
(histogram-with-curve "Gaussian (weight)"
           (replicate 5000 (gen [] (gaussian 150 20))) 
           (gen [x] (exp (score-gaussian x [150 20])))
           [80 220])

So one idea for how to model a person's height and weight would be to draw them individually from these distributions:

In [ ]:
(define person-v1
  (gen [] 
    (define height (gaussian 70 3))
    (define weight (gaussian 150 20))
    [height weight]))

But there's something wrong about this model: the height and weight are totally independent. In real life, a tall person is likely to weigh more; in our model, this is not the case.

Enter the _bivariate Gaussian_: in addition to a pair of means, it also takes a _covariance matrix_, a symmetric 2x2 matrix $\Sigma$, where $\sigma_{11}$ and $\sigma_{22}$ are the variances (squared standard deviations) of variables 1 and 2, but $\sigma_{12} = \sigma_{21}$ is the _covariance_ of the variables with one another. A positive covariance indicates direct correlation; a negative covariance indicates inverse correlation. Zero covariance means the variables are completely independent. (Note: in general, zero covariance does _not_ imply independence, but in a bivariate Gaussian model, it does.)

There are a number of ways to sample from a bivariate Gaussian. Below, we model the distribution as follows: first, we generate the first variable according to its mean and standard deviation (e.g., generate a height). Then, we figure out (based on how far that sample was from the mean, and the covariance), the expectation of the second variable (e.g., given the height we generated, what would we expect the weight to be?). Using the variance and covariance information, we can also calculate how much of a spread we expect to find around that expected value. Now that we have a new mean and covariance, we can sample the second variable (generate a weight). Here's the model:

In [ ]:
(define conditional-biv-gaussian-mean
  (gen [x1 [[mu1 mu2] [[sigma_11 sigma_12] [sigma_21 sigma_22]]]]
    (+ mu2 (* (- x1 mu1) (/ sigma_12 sigma_11)))))

(define conditional-biv-gaussian-variance
  (gen [x1 [[mu1 mu2] [[sigma_11 sigma_12] [sigma_21 sigma_22]]]]
    (/ (- (* sigma_11 sigma_22) (* sigma_12 sigma_12)) sigma_11)))

(define conditional-biv-gaussian-density
  (gen [x1 params]
    (gen [x]
    (exp (score-gaussian x [(conditional-biv-gaussian-mean x1 params)
                            (sqrt (conditional-biv-gaussian-variance x1 params))])))))

(define biv-gauss
  (gen [[mu1 mu2] [[sigma_11 sigma_12] [sigma_21 sigma_22]]]
    (with-explicit-tracer t
       ; draw the first variable normally 
       (define x1 (t "x1" gaussian mu1 (sqrt sigma_11)))
       ; calculate the new mean and variance of the second variable
       (define x2_mean
           (conditional-biv-gaussian-mean 
               x1
               [[mu1 mu2] [[sigma_11 sigma_12] [sigma_21 sigma_22]]]))
       (define x2_var
           (conditional-biv-gaussian-variance 
               x1
               [[mu1 mu2] [[sigma_11 sigma_12] [sigma_21 sigma_22]]]))
       ; draw the second variable
       (define x2 (t "x2" gaussian x2_mean (sqrt x2_var)))
       ; return both
       [x1 x2])))

In [ ]:
(clojure.core/require '[metaprob.compositional :as comp] :reload-all)

In [ ]:
(get (gen [x] 1) :generative-source)

In [ ]:
(infer :procedure comp/infer-apply :inputs [(gen [x] 1) [0.5] (make-top-level-tracing-context {} {})])

In [ ]:
(define person-v2
  (gen []
    ; 9 = 3^2, 400 = 20^2; we pass variance instead of standard deviation here
    ; setting covariance to 0 (instead of 40) would exactly recover the person-v1 model above.
    (biv-gauss [70 150] [[9 40] [40 400]])))
(define [height-adr weight-adr] (addresses-of (nth (infer :procedure person-v2 :inputs []) 1)))

Let's create a scatter plot to visualize the sorts of samples this generates:

In [ ]:
(custom-scatter-plot "Bivariate Gaussian"
  (replicate 1000 person-v2)
  "cross" "white" "blue" [[60 80] [90 210]])

The "shape" of this data is elliptical: there is a rotated ellipse in which the points are concentrated, with density decreasing as we move away from the center of the ellipse along either of its axes. We can see this better by plotting the actual probability density; to do so, we write an assessor function that exactly computes the probability density of a specific point. Graphing that function in a contour plot can help us better understand how the idea of a "bell curve" generalizes to two dimensions:

In [ ]:
(define biv-gaussian-density
  (gen [[mu1 mu2] [[s11 s12] [s21 s22]]]
    (gen [x y]
      (define x-from-mu (- x mu1))
      (define y-from-mu (- y mu2))
      (/
       (exp
        (/
          (- (+
              (/ (* x-from-mu x-from-mu) s11)
              (/ (* y-from-mu y-from-mu) s22))     
            (/ (* 2 s12 x-from-mu y-from-mu) (* s11 s22)))
          (* -2 (/ (- (* s11 s22) (* s12 s12)) (* s11 s22)))))
        (* 2 3.1415926 (sqrt (- (* s11 s22) (* s12 s12))))))))

(scatter-with-contours
  "Bivariate Gaussian"
  (replicate 100 person-v2); [] ;(replicate 100 (gen [] (biv-gauss [0 0] [[1/5 0] [0 1/5]])))
  (biv-gaussian-density [70 150] [[9 40] [40 400]])
  [[60 80] [90 210]])

In the plot, blue regions correspond to areas of low probability, and red regions to areas of high probability. Equiprobability contours are ellipses. The wider or taller the ellipses, the more uncertainty there is. The "off-center" rotation you see tells us that variables $x$ and $y$ are not independent: if we know one, it changes our guess about the other.

To get a feel for what the parameters of the bivariate Gaussian means, let's look at how the contour plots change as we change the parameters:

In [ ]:
; Changing the means moves the ellipse
(scatter-with-contours
  "Bivariate Gaussian"
  []
  (biv-gaussian-density [65 120] [[9 40] [40 400]])
  [[60 80] [90 210]])

In [ ]:
; Increasing sigma_11 increases the spread in the x direction.
(scatter-with-contours
  "Bivariate Gaussian"
  []
  (biv-gaussian-density [70 150] [[18 40] [40 400]])
  [[60 80] [90 210]])

In [ ]:
; Increasing sigma_22 increases the spread in the y direction.
(scatter-with-contours
  "Bivariate Gaussian"
  []
  (biv-gaussian-density [70 150] [[9 40] [40 800]])
  [[60 80] [90 210]])

In [ ]:
; Decreasing sigma_12 decreases the correlation between the variables, leading to a "flatter" ellipse
(scatter-with-contours
  "Bivariate Gaussian"
  []
  (biv-gaussian-density [70 150] [[9 0] [0 400]])
  [[60 80] [90 210]])

Note in this last plot that no matter what $x$ is, we induce the same distribution on $y$ (and vice versa).

Now, going back to our model of height and weight, suppose we know someone's weight and want to guess their height. We could use rejection sampling, but it's slow:

In [ ]:
; Height samples for someone whose weight is 175
(define conditional-samples
  (replicate 100 
    (gen []
      (rejection-sample 
        person-v2
        (gen [t] (< 174.5 (trace-value t weight-adr) 175.5))
        (gen [t] [(trace-value t height-adr) (trace-value t weight-adr)])))))

(histogram-with-curve
  "Exact posterior and rejection samples of height"
  (map (gen [[h w]] h) conditional-samples)
  (conditional-biv-gaussian-density 175 [[150 70] [[400 40] [40 9]]])
  [60 80])

;; (scatter-with-contours 
;;     "Posterior samples of height"
;;     conditional-samples
;;     (biv-gaussian-density [70 150] [[9 40] [40 400]])
;;     [[60 80] [174.4 175.6]])

In this histogram, we've also plotted the correct probability density for the posterior distribution. Generally, it is not tractable to compute exact posteriors—otherwise approximate inference techniques like the ones we've been exploring would be unnecessary. But for this simple case, of a bivariate Gaussian, it is possible, and we use the analytically computed curve to evaluate how well our sampling technique approximates the posterior distribution. 

We can try to fix the speed problem with sampling/importance resampling. If we use only one particle, we are essentially disregarding the condition and sampling from the prior:

In [ ]:
; Height samples for someone whose weight is 175
(define sir-samples-1-particle
  (replicate 100 
    (gen [] 
      (sample-importance-resample 
          person-v2
          (trace-set-value {} weight-adr 175)
          (gen [t] [(trace-value t height-adr) (trace-value t weight-adr)])
          1))))


(histogram-with-curve
  "Exact posterior and prior samples of height"
  (map (gen [[h w]] h) sir-samples-1-particle)
  (conditional-biv-gaussian-density 175 [[150 70] [[400 40] [40 9]]])
  [60 80])

;; (scatter-with-contours 
;;     "Prior samples of height"
;;     isr-samples-1-particle
;;     (biv-gaussian-density [70 150] [[9 40] [40 400]])
;;     [[60 80] [174.8 175.2]])

We can increase the number of particles and shift the distribution toward the true posterior, at a cost of spending more time per sample. See if you can play with the number of particles (below, 5) to achieve a good trade-off.

In [ ]:
; Height samples for someone whose weight is 175
(define sir-samples-multiple-particles
  (replicate 100 
    (gen [] 
      (sample-importance-resample 
          person-v2
          (trace-set-value {} weight-adr 175)
          (gen [t] [(trace-value t height-adr) (trace-value t weight-adr)])
          5))))


(histogram-with-curve
  "Exact posterior and approximate importance samples of height"
  (map (gen [[h w]] h) sir-samples-multiple-particles)
  (conditional-biv-gaussian-density 175 [[150 70] [[400 40] [40 9]]])
  [60 80])

;; (scatter-with-contours 
;;     "Approximate posterior samples of height"
;;     isr-samples-10-particles
;;     (biv-gaussian-density [70 150] [[9 40] [40 400]])
;;     [[60 80] [174.8 175.2]])

# 8.1 Custom inference procedures

In the case of the bivariate Gaussian, we happen to know exactly the mathematical form of the posterior. That is, for our person model, we can calculate the exact distribution that someone's height is drawn from when we know their weight. It is silly to use approximate algorithms when an exact one exists.

Metaprob is designed to be easily extensible, and to allow practitioners to encode any special knowledge like this directly into their models. It supports a class of user-defined procedures called "custom inference procedures," which specify not only a generative model, but also a custom "implementation" that overrides Metaprob's default behavior in the presence of target or intervention traces. In other words, we can specify custom behavior for when `(infer ...)` is called on our model.

In [ ]:
(clojure.core/require '[metaprob.context :refer :all])

In [ ]:
(clojure.core/use 'metaprob.context :reload-all)

In [ ]:
; We use `inf` to define a custom inference procedure. It takes in:
; -- a name
; -- a model
; -- an "implementation"
(define custom-biv-gauss
  (inf "biv-gauss" ; name
       biv-gauss   ; model
       ; implementation:
       ; accepts four arguments. 
       ;    1. inputs to the model (biv gaussian params)
       ;    2. an intervention trace
       ;    3. a target trace
       ;    4. a boolean `out?` that specifies whether to return an output
       ;       trace.
       ; returns three things:
       ;    1. the output of the model on these inputs
       ;    2. if out? is true, an output trace consistent with intervene and
       ;        target.
       ;    3. a score, the log of a number proportional to the _ratio_ between the true posterior density
       ;       and the density (at the sampled trace) of the distribution from which our implementation samples.
       ;       Because we are sampling from the exact posterior here, the ratio is 1, and so
       ;       we return log(1) == 0.
       (gen [params ctx]
        (define [[mu1 mu2] [[s11 s12] [s21 s22]]] params)
        ; read x1 from intervention or target trace,or, if it's not specified
        ; but x2 is in the target trace, sample it from the exact posterior.
        ; if neither x1 nor x2 are constrained, sample a fresh x1 just like
        ; the biv-gauss model does.
        (define x1-sample 
          (if (constrained? ctx "x1")
              (constrained-value ctx "x1")
              (gaussian 
                (if (targeted? ctx "x2") (conditional-biv-gaussian-mean (target-value ctx "x2") [[mu2 mu1] [[s22 s12] [s21 s11]]]) mu1)
                (sqrt (if (targeted? ctx "x2") (conditional-biv-gaussian-variance (target-value ctx "x2") [[mu2 mu1] [[s22 s12] [s21 s11]]]) s11)))))
         
         ; Either use the provided value for x2, or sample conditionally based on x1.
        (define x2-sample 
          (if (constrained? ctx "x2")
              (constrained-value ctx "x2")
              (gaussian (conditional-biv-gaussian-mean (or (intervene-value ctx "x1") x1-sample) params)
                        (sqrt (conditional-biv-gaussian-variance (or (intervene-value ctx "x1") x1-sample) params)))))
        
        (define score
          (cond 
              (and (targeted? ctx "x1") (targeted? ctx "x2"))
              (log ((biv-gaussian-density [mu1 mu2] [[s11 s12] [s21 s22]]) (target-value ctx "x1") (target-value ctx "x2")))
              (targeted? ctx "x1") (score-gaussian (target-value ctx "x1") [mu1 (sqrt s11)])
              (targeted? ctx "x2") (score-gaussian (target-value ctx "x2") [mu2 (sqrt s22)])
              true 0))
            
        ; Return values
        [[x1-sample x2-sample]
         (trace-set-value (trace-set-value {} "x1" x1-sample) "x2" x2-sample)
        score])))

Using this, sampling/importance resampling will "just work," no matter how many particles we use.

In [ ]:
; rewrite person-v2 using custom-biv-gauss instead of biv-gauss
(define person-v3
  (gen []
    (custom-biv-gauss [70 150] [[9 40] [40 400]])))
(define [height-adr weight-adr] (addresses-of (nth (infer :procedure person-v3) 1)))


; Height samples for someone whose weight is 170
(define sir-samples-1-particle
  (replicate 100 
    (gen [] 
      (sample-importance-resample 
          person-v3
          (trace-set-value {} weight-adr 175)
          (gen [t] [(trace-value t height-adr) (trace-value t weight-adr)])
          1))))


(histogram-with-curve
  "Exact posterior with fast exact samples"
  (map (gen [[h w]] h) sir-samples-1-particle)
  (conditional-biv-gaussian-density 175 [[150 70] [[400 40] [40 9]]])
  [60 80])

## 8.2 Using the bivariate Gaussian as part of another model

We can use the bivariate Gaussian to help implement a new model. We will model the following strange situation. Suppose we go to the circus and see an act in which two brothers have a circus act; they pretend to be one very tall person by standing on top of one another and wearing a large coat. From our seats, we can estimate (perhaps with some error) the total height of both brothers, but would like to infer their individual heights. Here is the model:

In [ ]:
(define circus-brothers
  (gen []
    (define [h1 h2]
      (biv-gauss [70 70] [[9 5] [5 9]]))
    (define total-height
        (gaussian (+ h1 h2) 3))
    total-height))

(define [h1-adr h2-adr total-adr] (addresses-of (nth (infer :procedure circus-brothers) 1)))

In [ ]:
(scatter-with-contours
  "Prior distribution of brothers' heights"
  []
  (biv-gaussian-density [70 70] [[9 5] [5 9]])
  [[60 80] [60 80]])

In [ ]:
(define observed-height 155)
(scatter-with-contours
  "Posterior distribution of brothers' heights"
  []
  (biv-gaussian-density [(+ 17.02 (* 0.378 observed-height)) (+ 17.02 (* 0.378 observed-height))] [[3.7 -0.2966] [-0.2966 3.7]])
  [[60 80] [60 80]])

This shape should make some sense: because 155 is a bit higher than what we'd ordinarily expect, the distribution has moved up and to the right. It's also straightened out: originally, knowing that one brother was taller would also make us think the second brother was taller, because brothers tend to have similar heights. But now, given that their total height is constrained to be somewhere near 155, learning that one brother is very tall would lead us to believe the other brother must be a bit shorter (and vice versa).

Let's plot some samples, using all our techniques for sampling from a conditional.

In [ ]:
(define extract-heights
  (gen [t] [(trace-value t h1-adr) (trace-value t h2-adr)]))

; From the prior
(define prior-samples
  (replicate 100 
    (gen [] 
      (sample-importance-resample
          circus-brothers
          (trace-set-value {} total-adr observed-height)
          extract-heights
          1))))

(scatter-with-contours
  "Prior samples of brothers' heights"
  prior-samples
  (biv-gaussian-density [(+ 17.02 (* 0.378 observed-height)) (+ 17.02 (* 0.378 observed-height))] [[3.7 -0.2966] [-0.2966 3.7]])
  [[60 80] [60 80]])

In [ ]:
; From the approximate posterior
(define approx-posterior-samples
  (replicate 100 
    (gen [] 
      (sample-importance-resample
          circus-brothers
          (trace-set-value {} total-adr observed-height)
          extract-heights
          10))))

(scatter-with-contours
  "Approx. posterior samples of brothers' heights"
  approx-posterior-samples
  (biv-gaussian-density [(+ 17.02 (* 0.378 observed-height)) (+ 17.02 (* 0.378 observed-height))] [[3.7 -0.2966] [-0.2966 3.7]])
  [[60 80] [60 80]])

In [ ]:
; From the posterior but slowly
(define posterior-samples
  (replicate 100 
    (gen [] 
      (rejection-sample 
          circus-brothers
          (gen [t]
               (< -0.5
                  (- observed-height (trace-value t total-adr))
                  0.5))
          extract-heights))))

(scatter-with-contours
  "Posterior samples of brothers' heights"
  posterior-samples
  (biv-gaussian-density [(+ 17.02 (* 0.378 observed-height)) (+ 17.02 (* 0.378 observed-height))] [[3.7 -0.2966] [-0.2966 3.7]])
  [[60 80] [60 80]])

One technique we can use to get more accurate samples than sampling/importance resampling in less time than rejection sampling is to use "Markov Chain Monte Carlo" (MCMC) methods. These methods involve starting with a single sample and iteratively changing it, in such a way that eventually, each successive iteration yields a sample from the posterior. There are many ways to do this, but below, we use an algorithm called Single-Site Metropolis Hastings, which (in our case) will iteratively propose changes to one brother's height at a time, then randomly deciding to either accept or reject the proposal. (The probability with which a proposal is accepted depends on a couple factors, including how likely our constraint is given the proposal.)

In [ ]:
(define initial-sample
  ((infer :procedure circus-brothers
         :target-trace (trace-set-value {} total-adr 155)) 1))

(define mh-traj
    (clojure.core/reduce
      (gen [samples _] 
           (clojure.core/cons
               (single-site-metropolis-hastings-step
                           circus-brothers []
                           (first samples) (list total-adr))
       samples))
  (list initial-sample)
  (range 100)))

(scatter-with-contours
  "Posterior samples of brothers' heights"
  (clojure.core/reverse (map extract-heights mh-traj))
  (biv-gaussian-density [(+ 17.02 (* 0.378 observed-height)) (+ 17.02 (* 0.378 observed-height))] [[3.7 -0.2966] [-0.2966 3.7]])
  [[60 80] [60 80]])

Click Play in the animated plot below to view the trajectory. It flashes red whenever the trajectory stays stationary because of a rejected proposal.

In [ ]:
(mh-animation "Posterior samples of brothers' heights"
              (clojure.core/reverse (map extract-heights mh-traj))
              [[60 80] [60 80]])

### Challenge Exercise C: Custom inference for circus brothers <a name="ex-custombrothers"></a>

Implement a custom inference procedure for `circus-brothers`, using the fact that conditioned on a certain observed total height $h$, the brothers' individual heights are drawn from a bivariate Gaussian:
- with means $\mu_1 = \mu_2 = 17.02 + 0.378h$, 
- with each height having marginal variance $3.7$, and
- with the covariance between the two heights equal to $-0.2966$. 

Plot importance samples and MH samples from this model.

In [ ]:
; Your solution here

In [ ]:
; Solution

; [To be added]